## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `BETA`. 

In [20]:
import mne

# tarefa 1
data = mne.read_epochs("../preprocessing_beta/beta.fif")

Reading C:\Users\anaca\OneDrive\Documentos\GitHub\ssvep\SSVEP\classificador\..\preprocessing_beta\beta.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1462.89 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


C:\Users\anaca\AppData\Local\Temp\ipykernel_12320\2329412296.py:4: RuntimeWarning: This filename (../preprocessing_beta/beta.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data = mne.read_epochs("../preprocessing_beta/beta.fif")


In [21]:
# TAREFA 2
threshold = 0

# entendimento dos dados
print(data.get_data().shape)
data

(160, 64, 750)


Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 1.463 s
Baseline,off


In [26]:
# TAREFA 3

import numpy as np

targets = [float(item) for item in data.event_id.keys()]
print("Possíveis alvos:", targets)

features = list()
for i in range(len(data)):
    sample = list()
    for target in targets:
        fmin = target - threshold
        fmax = target + threshold
        sample.append(data[i].compute_psd(method='multitaper', fmin=fmin,
                                          fmax=fmax, verbose=False))
    features.append(sample)

X = np.array(features)
print("Formato dos dados calculados:", X.shape)

Possíveis alvos: [8.0, 8.2, 8.4, 8.6, 8.799999999999999, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.600000000000001, 12.8, 13.0, 13.200000000000001, 13.4, 13.600000000000001, 13.8, 14.0, 14.200000000000001, 14.4, 14.600000000000001, 14.8, 15.0, 15.200000000000001, 15.4, 15.600000000000001, 15.8]


ValueError: zero-size array to reduction operation minimum which has no identity

In [25]:
# organizando os dados

X = X.reshape(X.shape[0], X.shape[1], X.shape[-1])
print("Formato padronizado dos dados com PSD calculado:", X.shape)

NameError: name 'X' is not defined

In [ ]:
# TAREFA 4

# aplicando a característica de "maior valor"
max_values = np.max(X, axis=-1)
max_values.shape

In [ ]:
# TAREFA 5

y = np.load("../datasets/avi/multi/labels_multi.npy")

# entendimento dos dados
print(y, y.shape, targets)

# classificação pelo "voto" do maior valor
i_max = max_values.argmax(axis=-1)
hits = [1 for i in range(len(i_max)) if targets[i_max[i]] == y[i]]
acc = 100 * sum(hits) / len(y)
print(f'\nPorcentagem de acerto: {acc:.2f}%')